# Setup

In [ ]:
import os
import logging as log
from lisa.utils import setup_logging
setup_logging(level=log.ERROR)

from lisa.trace import Trace
from lisa.wa import WAOutput
from lisa.stats import Stats
from lisa.datautils import series_mean
from pandas import DataFrame
import pandas as pd
import scipy as sp
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import holoviews as hv
from holoviews import opts
from bokeh.themes import built_in_themes
from tabulate import tabulate

from holoviews.operation.datashader import datashade, rasterize
from holoviews.operation import decimate

from wp.notebook import WorkloadNotebookAnalysis, trim_wa_path, ptable

hv.extension('bokeh')
hv.renderer('bokeh').theme = built_in_themes['dark_minimal']
hv.renderer('bokeh').webgl = True
pio.templates.default = "plotly"
pio.templates.default = "plotly_dark"

color_cycle = hv.Cycle(['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'])

opts.defaults(
    opts.Curve(tools=['hover'], show_grid=True, color=color_cycle, muted_alpha=0),
    opts.Table(bgcolor='black')
)

## Runs

In [ ]:
ft = WorkloadNotebookAnalysis('/home/kajpuc01/power/pixel6/fortnite/', [
    'fortnite_product_baseline_10_2007',
    'fortnite_product_arm_vh_sched_debug_10_2007'
])

ft.show()

# FPS

## FPS line plots

In [ ]:
ft.plot.results_line(['fps', 'total_frames', 'janks_pc', 'missed_vsync'], height=500, width=800, title='FPS metrics per iteration')

## FPS bar plots

In [ ]:
ft.plot.results_bar(['fps', 'total_frames', 'janks_pc', 'missed_vsync'], height=600, columns=4, title='gmean FPS metrics')

## FPS over time

In [ ]:
# TODO

# Overutilized

## Line plot

In [ ]:
ft.plot.overutilized_line()

# Power meter

## Line plot across iterations

In [ ]:
ft.plot.power_meter_line()

## Bar plot - gmean

In [ ]:
ft.plot.power_meter_bar()

# Capacity

In [ ]:
ft.load_combined_analysis('capacity.pqt', allow_missing=True)

display(ft.analysis['capacity'])

## Line plot

In [ ]:
ds = hv.Dataset(ft.analysis['capacity'].query("iteration == 7 and cpu == 6"), ['time_it', hv.Dimension('wa_path', values=ft.wa_paths), 'iteration', 'cpu'], ['capacity'])
layout_cap = ds.to(hv.Curve, 'time_it', 'capacity').overlay('wa_path').opts(width=2100, height=600)
layout_cap

# uclamp

In [ ]:
def postprocess_uclamp_updates(df):
    return df.query("task != '<unknown>'")

ft.load_combined_analysis('uclamp_updates.pqt', postprocess=postprocess_uclamp_updates, allow_missing=True)

display(ft.analysis['uclamp_updates'])

## Per-task uclamp over time

In [ ]:
ds = hv.Dataset(ft.analysis['uclamp_updates'], ['time_it', hv.Dimension('wa_path', values=ft.wa_paths), 'iteration', 'uclamp_id', 'task'], ['value'])
layout = ds.to(hv.Curve, 'time_it', 'value').overlay('wa_path').opts(shared_axes=False, title='Per-task uclamp over time')

layout.opts(
    opts.Curve(height=600, width=1600, interpolation='steps-post', framewise=True)
)
layout

## Frequency & uclamp over time

In [ ]:
ft.load_combined_analysis('freqs.pqt')

def adjust_df_trace_time(it, path, df):
    df['Time'] = df['Time'] - ft.traces[path][it].start
    return df

ft.analysis['freqs_adj'] = pd.concat([adjust_df_trace_time(tags[0], tags[1], df) for tags, df in ft.analysis['freqs'].groupby(["iteration", 'wa_path'])])
ft.analysis['freqs_adj']['frequency_scaled'] = ft.analysis['freqs_adj']['frequency'] / 100000

display(ft.analysis['freqs_adj'])

In [ ]:
layout = hv.Dataset(ft.analysis['uclamp_updates'].query("uclamp_id == 0"), ['Time', hv.Dimension('wa_path', values=ft.wa_paths), 'iteration'], ['value']).select(
    task='UnityMain'
).to(hv.Curve, 'Time', 'value').overlay('wa_path').opts(shared_axes=False, title='CPU frequency & per-task uclamp over time')
layout *= hv.Dataset(ft.analysis['freqs_adj'], ['Time', hv.Dimension('wa_path', values=ft.wa_paths), 'cpu', 'iteration'], ['frequency_scaled']).to(
    hv.Curve, 'Time', 'frequency_scaled'
).overlay('wa_path').opts(shared_axes=False)

layout.opts(
    opts.Curve(height=600, width=1600, interpolation='steps-post', framewise=True)
)
layout

# Task placement (activations)

In [ ]:
# select all tasks specified in the config by default
plot_tasks = ft.config['processor']['important_tasks']['fortnite'].get()
# override to select fewer tasks - must be a subset of the above
plot_tasks = ['GameThread', 'RHIThread', 'RenderThread 0', 'surfaceflinger', 'mali_jd_thread', 'RenderEngine']

def postprocess_task_activations_stats_cluster_df(df):
    return df.query("comm in @plot_tasks").reset_index(drop=True)

ft.load_combined_analysis('task_activations_stats_cluster.pqt', postprocess=postprocess_task_activations_stats_cluster_df)
ft.analysis['task_activations_stats_cluster_melt'] = pd.melt(ft.analysis['task_activations_stats_cluster'], id_vars=['kernel', 'wa_path', 'iteration', 'cluster', 'comm'], value_vars=['count', 'duration'])
ft.analysis['task_activations_stats_cluster_melt']

## Overview

### Line plots - counts

In [ ]:
for task, task_df in ft.analysis['task_activations_stats_cluster'].groupby('comm'):
    ft.plot_lines_px(task_df, x='iteration', y='count', color='wa_path', facet_col='cluster', facet_col_wrap=3, height=500, scale_y=True, title=f'Activations of {task} per cluster across iterations')

### Line plots - durations

In [ ]:
for task, task_df in ft.analysis['task_activations_stats_cluster'].groupby('comm'):
    ft.plot_lines_px(task_df, x='iteration', y='duration', color='wa_path', facet_col='cluster', facet_col_wrap=3, height=500, scale_y=True, title=f'Activation durations of {task} per cluster across iterations')

### Bar plot - counts

In [ ]:
ft.summary['activations_stats_count'] = ft.plot_gmean_bars(
    ft.analysis['task_activations_stats_cluster_melt'].query("variable == 'count'").reset_index(drop=True),
    x='cluster', facet_col='comm', facet_col_wrap=3, title='Gmean task activation counts', height=1000, include_columns=['cluster'], order_cluster=True
)

### Bar plot - durations

In [ ]:
ft.summary['activations_stats_durations'] = ft.plot_gmean_bars(
    ft.analysis['task_activations_stats_cluster_melt'].query("variable == 'duration'").reset_index(drop=True),
    x='cluster', facet_col='comm', facet_col_wrap=3, title='Gmean task activation durations', height=1000, include_columns=['cluster'], order_cluster=True
)